<a href="https://colab.research.google.com/github/sunnatillo91/ai-datascience/blob/main/05_ML_04_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. MACHINE LEARNING

## 4-QADAM. Machine Learning

In [1]:
import pandas as pd
import numpy as np
import sklearn  # scikit-leaarn kutubxonasi

# Onlayn dataset joylashgan manzilni ko'rsatamiz
url = 'http://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true'
df = pd.read_csv(url)

- Ma'lumotlarni train va testga ajratamiz

In [10]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train= train_set.drop("median_house_value", axis=1)
y = train_set["median_house_value"].copy()

X_num=X_train.drop("ocean_proximity", axis=1)

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

#bizga kerak ustunlar indekslari
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_room = True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]

# Pipeline da sonlar

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

num_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy="median")),
  ('attribs_adder', CombinedAttributesAdder()),
  ('std_scaler', StandardScaler()),
])

num_pipeline.fit_transform(X_num)

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.2117846 ],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222,  0.34218528],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.66165785],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568,  0.99951387],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.79086209],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429,  1.69520292]])

#FullPipeline, Pipeline da matnlar bilan ishlash

In [13]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
  ("num", num_pipeline, num_attribs),
  ("cat", OneHotEncoder(), cat_attribs),
])

In [14]:
X_prepared = full_pipeline.fit_transform(X_train)

In [15]:
X_prepared[0:5,:]

array([[ 1.27258656, -1.3728112 ,  0.34849025,  0.22256942,  0.21122752,
         0.76827628,  0.32290591, -0.326196  , -0.17491646,  0.05137609,
        -0.2117846 ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.70916212, -0.87669601,  1.61811813,  0.34029326,  0.59309419,
        -0.09890135,  0.6720272 , -0.03584338, -0.40283542, -0.11736222,
         0.34218528,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.44760309, -0.46014647, -1.95271028, -0.34259695, -0.49522582,
        -0.44981806, -0.43046109,  0.14470145,  0.08821601, -0.03227969,
        -0.66165785,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.23269811, -1.38217186,  0.58654547, -0.56148971, -0.40930582,
        -0.00743434, -0.38058662, -1.01786438, -0.60001532,  0.07750687,
         0.78303162,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-0.10855122,  0.5320839 ,  1

### LinearRegression

In [16]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_prepared, y)

LinearRegression()

In [17]:
test_data = X_train.sample(10)
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
19469,-120.99,37.68,28.0,3269.0,647.0,1595.0,617.0,2.2336,INLAND
12245,-116.97,33.74,20.0,3674.0,792.0,1498.0,758.0,2.2161,INLAND
1528,-122.07,37.89,38.0,2139.0,343.0,809.0,340.0,5.5636,NEAR BAY
4425,-118.24,34.07,27.0,223.0,80.0,249.0,82.0,1.6136,<1H OCEAN
16802,-122.42,37.66,41.0,2189.0,414.0,1063.0,409.0,4.7361,NEAR OCEAN
16643,-120.65,35.32,20.0,626.0,212.0,3574.0,261.0,1.0298,NEAR OCEAN
2705,-115.59,32.85,20.0,1608.0,274.0,862.0,248.0,4.8750,INLAND
11090,-117.89,33.82,24.0,1268.0,210.0,700.0,224.0,5.0605,<1H OCEAN
1473,-122.03,37.97,45.0,1613.0,338.0,865.0,336.0,3.2500,NEAR BAY
16991,-122.28,37.55,17.0,4199.0,629.0,2020.0,630.0,6.1228,NEAR BAY


In [18]:
test_labels = y.loc[test_data.index]
test_labels

19469    112700.0
12245     76900.0
1528     268800.0
4425     137500.0
16802    302600.0
16643    300000.0
2705      90800.0
11090    216200.0
1473     151100.0
16991    375700.0
Name: median_house_value, dtype: float64

In [20]:
test_data_prepared = full_pipeline.transform(test_data)
test_labels_predictions = lin_reg.predict(test_data_prepared)
test_labels_predictions

array([103837.15675803, 111187.03356009, 295432.28483255, 151746.7469363 ,
       292165.21011541,  37244.73447686, 158414.24968651, 242629.04893267,
       209440.29695469, 298662.78275835])

In [21]:
pd.DataFrame({"Actual": test_labels, "Predicted": test_labels_predictions})

,Actual,Predicted
19469,112700.0,103837.156758
12245,76900.0,111187.033560
1528,268800.0,295432.284833
4425,137500.0,151746.746936
16802,302600.0,292165.210115
16643,300000.0,37244.734477
2705,90800.0,158414.249687
11090,216200.0,242629.048933
1473,151100.0,209440.296955
16991,375700.0,298662.782758
